## Apache kafka - wprowadzenie

Apache kafka to ,,serwer'' oraz ,,baza danych''.

Domyślnym adresem naszego serwera jest `broker:9092`

Nasza baza danych będzie posługiwała się tabelami, które w Apache Kafka nazywamy `topicami`.

Zarządzanie Kafką odbywa się z wykorzystaniem skryptów. W naszym przypadku będą to skrypty  `*.sh`.


### Sprawdź listę topiców

Pamiętaj aby przejść do katalogu domowego.

```bash
cd ~ 
kafka/bin/kafka-topics.sh --list --bootstrap-server broker:9092
```

### Utwórz nowy topic o nazwie `mytopic`

```bash
kafka/bin/kafka-topics.sh --create --topic mytopic  --bootstrap-server broker:9092
```

### Utwórz producenta 

Ten skrypt pozwoli Ci wprowadzać eventy ręcznie przez terminal.
Opcje `--property` są dodatkowe i wprowadzone tylko do analizy przeprowadzanej w tym przykładzie.

```bash
kafka/bin/kafka-console-producer.sh --bootstrap-server broker:9092 --topic mytopic --property "parse.key=true" --property "key.separator=:"
```

### Consument w Sparku

Otwórz nowy terminal w miejscu gdzie znajduje się plik test_key_value.py i uruchom program Consumenta na Sparku.

```python
from pyspark.sql import SparkSession

KAFKA_BROKER = 'broker:9092'
KAFKA_TOPIC = 'mytopic'

spark = SparkSession.builder.getOrCreate()

spark.sparkContext.setLogLevel("WARN")

df = (spark.readStream.format("kafka")
      .option("kafka.bootstrap.servers", KAFKA_BROKER)
      .option("subscribe", KAFKA_TOPIC)
      .option("startingOffsets", "earliest")
      .load()
     )
# to co trafia z kafki przychodzi w postaci binarnej więc trzeba przetworzyć to na string

(df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
 .writeStream
 .format("console")
 .outputMode("append")
 .start()
 .awaitTermination()
)
```


Pamiętaj, iż Apache Spark nie posiada domyślnego konektora do Kafki dlatego uruchom proces poprzez komendę spark-submit i pobierz odpowiedni pakiet w scali.

```bash
spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0  test_key_value.py
```

5. W terminalu z uruchominionym producentem wpisz teskt w postaci: 
```bash
jan:45
alicja:20
```
i sprawdz co pojawia się w oknie aplikacji consumenta.

Po zakończeniu pokazu ctrl+c zamyka zarówno okno producenta jak i aplikacje sparkową

## Apache Kafka - Zadanie 

1. Sprawdź listę tematów udostępnionych na serwerze Kafki `broker:9092`
2. Dodaj topic o nazwie `streaming##` gdzie w miejsce ## wstaw swój numer przydzielony na początku zajęć. 
3. Ponownie sprawdź listę tematów weryfikująć czy nowy teamt został dodany do listy.
4. Otwórz nowy terminal na notatniku i utwórz producenta w pliku `producer.py`
    - ustal nazwę topicu 
    - ustal bieżący czas i dane w postaci słownika z trzema kluczami i  wartościami: 
        - klucz_1 == `time` i przypisuje bierzącą datę
        - klucz_2 == `id` i wybiera losową wartość z listy `["a", "b", "c", "d"]`
        - kliucz_3  ==  `values` - zwraca losową wartość z zakresu 0, 1000


```python
import json
import random
import sys
from datetime import datetime
from time import sleep

from kafka import KafkaProducer

SERVER = "broker:9092"
# USTAL NAZWE TOPICU ZGODNIE Z TWOIM NUMEREM SRODOWISKA
TOPIC = "streaming##"
producer = KafkaProducer(
        bootstrap_servers=[SERVER],
        value_serializer=lambda x: json.dumps(x).encode("utf-8"),
    )
    
try:
    while True:

        # t = aktualna data i czas       
        t = ...
            
        message = { pass } 
            
            
        producer.send(TOPIC, value=message)
        sleep(1)

except KeyboardInterrupt:
    producer.flush()
    producer.close()
```


Uruchom nowego consumera w terminalu: 

```bash
kafka/bin/kafka-console-consumer.sh --bootstrap-server broker:9092 --topic streaming## --from-beginning
```